In [1]:
import os
import numpy as np
import pandas as pd

## 导入数据

In [2]:
sample_text_encode = pd.read_csv("./bert/sample_text_encode.csv", header=None)

In [3]:
labels = []
with open('./bert/train_label.csv', encoding = 'utf-8-sig') as f:
    while True:
        line = f.readline()
        if line == "":
            break
        line = line.strip('\n')
        labels.append(line)
f.close
labels = labels[1:]

20211

In [4]:
labels = [int(i) for i in labels]
y = np.array(labels)
x = np.array(sample_text_encode)

del sample_text_encode, labels

In [5]:
n = len(y)
import random
index = random.sample(range(n),n)
index_test = index[:1000]
index_train = index[1000:]
x_train = x[index_train]
y_train = y[index_train]
x_test = x[index_test]
y_test = y[index_test]

In [ ]:
!pip3 install optuna

In [6]:
# 转换为PyTorch张量
import torch
X_tensor = torch.tensor(x_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.long)

In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# 分割数据集为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=64) 

# 创建训练和验证的数据加载器
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [9]:
X1_tensor = torch.tensor(x_test, dtype=torch.float32)
y1_tensor = torch.tensor(y_test, dtype=torch.long)

In [10]:
val_dataset1 = TensorDataset(X1_tensor, y1_tensor)
val_loader1 = DataLoader(val_dataset1, batch_size=64, shuffle=False)

## 训练神经网络

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes, output_size):
        super(MLP, self).__init__()
        # 初始化权重
        self.initializer = nn.init.KaimingNormal()
        
        # 第一个隐藏层
        self.layers = [nn.Linear(input_size, hidden_layer_sizes[0]), nn.ReLU()]
        # 添加更多的隐藏层
        for i in range(1, len(hidden_layer_sizes)):
            self.layers.extend([
                nn.Linear(hidden_layer_sizes[i-1], hidden_layer_sizes[i]),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_layer_sizes[i])
            ])
        # 最后一个隐藏层到输出层
        self.layers.append(nn.Linear(hidden_layer_sizes[-1], output_size))
        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        x = self.model(x)
        return x

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                self.initializer(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

In [26]:
# 创建优化函数
def optimize(trial):
    # 定义超参数搜索空间
    input_size = 768  # 假设的输入特征大小
    hidden_layer_sizes = [trial.suggest_int('h1', 10, 200),
                         trial.suggest_int('h2', 10, 200),
                         trial.suggest_int('h3', 10, 200)]
    output_size = 2  # 类别数
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])
    batch_size = trial.suggest_int('batch_size', 32, 256, step=32)

    # 创建模型
    model = MLP(input_size, hidden_layer_sizes, output_size)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # model.to('cuda' if torch.cuda.is_available() else 'cpu')

    # 创建优化器
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    else:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # 训练模型
    # 训练时，设置模型为训练模式
    model.train()
    for epoch in range(10):  # 假设训练10个epoch
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data.to(device))
            loss = nn.CrossEntropyLoss()(output, target.to(device))
            loss.backward()
            optimizer.step()

    # 评估模型
    model.eval()
    # 在评估循环中，模型不会应用Dropout
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for data, target in val_loader:
            predictions = model(data.to(device)).argmax(dim=1)
            total_correct += (predictions == target.to(device)).sum().item()
            total_samples += target.size(0)
    accuracy = total_correct / total_samples

    # 返回验证集上的准确率
    return accuracy

In [ ]:
!pip3 install cmaes

In [27]:
# from optuna import create_study, CmaEsSampler
# from optuna.samplers import CmaEsSampler  # 确保导入了 CmaEsSampler
import optuna

# 创建Optuna研究对象
study = optuna.create_study(direction='maximize')
# sampler = CmaEsSampler(warn_independent_sampling=False)
# study = optuna.create_study(direction='maximize', sampler=sampler)

# 运行优化
study.optimize(optimize, n_trials=100)

# 输出最佳超参数
print(f'Best parameters found: {study.best_trial.params}')

[I 2024-03-02 08:21:57,002] A new study created in memory with name: no-name-2399bb08-a58e-490a-84b3-3ac01d476a43
[I 2024-03-02 08:21:59,798] Trial 0 finished with value: 0.5572594607964383 and parameters: {'h1': 694, 'h2': 434, 'h3': 109, 'dropout_rate': 0.25906861233345113, 'learning_rate': 0.00019616184780627278, 'optimizer': 'SGD', 'batch_size': 256}. Best is trial 0 with value: 0.5572594607964383.
[I 2024-03-02 08:22:03,077] Trial 1 finished with value: 0.8206777145683898 and parameters: {'h1': 721, 'h2': 409, 'h3': 215, 'dropout_rate': 0.026879509610615693, 'learning_rate': 0.0002847470474272649, 'optimizer': 'Adam', 'batch_size': 128}. Best is trial 1 with value: 0.8206777145683898.
[I 2024-03-02 08:22:06,285] Trial 2 finished with value: 0.8083106604006925 and parameters: {'h1': 752, 'h2': 346, 'h3': 259, 'dropout_rate': 0.28131795956310746, 'learning_rate': 0.0021159682449778294, 'optimizer': 'Adam', 'batch_size': 160}. Best is trial 1 with value: 0.8206777145683898.
[I 2024-0

Best parameters found: {'h1': 643, 'h2': 409, 'h3': 469, 'dropout_rate': 0.3333532600446642, 'learning_rate': 0.00025435001897031634, 'optimizer': 'Adam', 'batch_size': 32}


### 保留训练的最优模型的参数

In [17]:
# 获取最佳超参数
best_params = study.best_trial.params
# 获取最佳分数
best_score = study.best_trial.value
# 输出最佳分数
print(f'Best score found: {best_score}')

Best score found: 0.8360128617363344


In [18]:
import json
best_params = study.best_trial.params
with open('best_params_h3_836.json', 'w') as f:
    json.dump(best_params, f)

## 用最优模型参数给出最优模型

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes, output_size,dropout_rate=0.0, learning_rate=0.01, optimizer='adam', batch_size=64):
        super(MLP, self).__init__()
        # 初始化权重
        self.initializer = nn.init.kaiming_normal_
        # 设置优化器
        if optimizer == 'adam':
            self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        elif optimizer == 'sgd':
            self.optimizer = optim.SGD(self.parameters(), lr=learning_rate)
        # 设置批量大小
        self.batch_size = batch_size
        
        # 第一个隐藏层
        layers = [nn.Linear(input_size, hidden_layer_sizes[0])]     
        # 添加ReLU激活函数
        layers.append(nn.ReLU())
        
        # 添加更多的隐藏层和批归一化层
        for i in range(1, len(hidden_layer_sizes)):
            layers.extend([
                nn.Linear(hidden_layer_sizes[i-1], hidden_layer_sizes[i]),
                nn.BatchNorm1d(hidden_layer_sizes[i]),
                nn.ReLU()
            ])
        
        # 最后一个隐藏层到输出层
        layers.append(nn.Linear(hidden_layer_sizes[-1], output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                self.initializer(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)


In [12]:
# 创建一个MLP模型实例
input_size = 768  # 输入特征维度
hidden_layer_sizes = [188, 158, 79]  # 假设有3个隐藏层，分别有188, 158, 79个神经元
output_size = 2  # 假设是2分类问题
dropout_rate = 0.08290526156007141
learning_rate = 0.00020148404515308536
optimizer ="Adam"
batch_size = 96
model = MLP(input_size, hidden_layer_sizes, output_size, dropout_rate, learning_rate, optimizer, batch_size)

In [13]:
import torch.optim as optim

# 假设您已经有了模型、数据加载器和损失函数
# model = MLP(input_size, hidden_layer_sizes, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
accuracies = []
for epoch in range(20):
    model.train()  # 设置模型为训练模式
    running_loss = 0.0
    for inputs, labels in train_loader:  # 假设train_loader是训练数据的DataLoader
        optimizer.zero_grad()  # 清空梯度
        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 参数更新
        
        running_loss += loss.item()
    
    # 验证模型
    model.eval()  # 设置模型为评估模式
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证时不计算梯度
        for inputs, labels in val_loader:  # 假设val_loader是验证数据的DataLoader
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # 打印统计信息
    print(f'Epoch {epoch+1}/{20}, Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')
    accuracies.append(100 * correct / total)


Epoch 1/20, Loss: 0.4561, Val Loss: 0.4161, Accuracy: 81.68%
Epoch 2/20, Loss: 0.3819, Val Loss: 0.3924, Accuracy: 82.20%
Epoch 3/20, Loss: 0.3489, Val Loss: 0.3964, Accuracy: 82.10%
Epoch 4/20, Loss: 0.3209, Val Loss: 0.3936, Accuracy: 81.79%
Epoch 5/20, Loss: 0.2910, Val Loss: 0.4147, Accuracy: 80.72%
Epoch 6/20, Loss: 0.2613, Val Loss: 0.4094, Accuracy: 82.20%
Epoch 7/20, Loss: 0.2276, Val Loss: 0.4456, Accuracy: 80.20%
Epoch 8/20, Loss: 0.1987, Val Loss: 0.4298, Accuracy: 82.33%
Epoch 9/20, Loss: 0.1789, Val Loss: 0.5008, Accuracy: 80.85%
Epoch 10/20, Loss: 0.1546, Val Loss: 0.4803, Accuracy: 81.76%
Epoch 11/20, Loss: 0.1305, Val Loss: 0.5097, Accuracy: 82.72%
Epoch 12/20, Loss: 0.1030, Val Loss: 0.5611, Accuracy: 82.25%
Epoch 13/20, Loss: 0.0936, Val Loss: 0.5631, Accuracy: 81.08%
Epoch 14/20, Loss: 0.0711, Val Loss: 0.6040, Accuracy: 81.39%
Epoch 15/20, Loss: 0.0766, Val Loss: 0.6652, Accuracy: 79.86%
Epoch 16/20, Loss: 0.0680, Val Loss: 0.6141, Accuracy: 82.07%
Epoch 17/20, Loss

In [14]:
# 算平均准确率
average_accuracy = sum(accuracies) / len(accuracies)
average_accuracy

81.54696851418161

### 保存模型

In [15]:
# 在训练结束后保存模型
torch.save(model.state_dict(), 'best_model_state_dict.pth')

### 测试保存的模型是否可用

In [ ]:
# 创建模型实例（确保模型架构与训练时相同）
# 创建一个MLP模型实例
input_size = 768  # 输入特征维度
hidden_layer_sizes = [188, 158, 79]  # 假设有3个隐藏层，分别有188, 158, 79个神经元
output_size = 2  # 假设是2分类问题
dropout_rate = 0.08290526156007141
learning_rate = 0.00020148404515308536
optimizer ="Adam"
batch_size = 96
model1 = MLP(input_size, hidden_layer_sizes, output_size, dropout_rate, learning_rate, optimizer, batch_size)

# 加载保存的模型参数
model1.load_state_dict(torch.load('./bert/best_model_state_dict.pth'))

In [123]:
# 验证模型
model1.eval()  # 设置模型为评估模式
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():  # 在验证时不计算梯度
    for inputs, labels in val_loader1:  # 假设val_loader是验证数据的DataLoader
        outputs = model1(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 打印统计信息
print(f' Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')

 Val Loss: 0.2228, Accuracy: 81.90%
